# Fire Damage Estimation

Table of Contence?

Project introduction

Define goal

Mention first steps

In [1]:
# NOTE: Make sure to clean library selection at the end and remove any that did not get used

# importing libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
# or
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score 
from sklearn.decomposition import PCA

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import SplineTransformer